In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("spotify_millsongdata.csv")

In [3]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [4]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [5]:
df.shape

(57650, 4)

In [6]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [7]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [8]:
df.head(10)

,artist,song,text
0,Robbie Williams,Hot Fudge,"Yeah! \r\n \r\nQueen Bitch, eat the rich \r..."
1,Joni Mitchell,Ray's Dad's Cadillac,Ray's Dad's Cadillac \r\nRollin' past the rin...
2,Imperials,Stand By The Power,"There is a power, \r\nAbsolute authority that..."
3,The Script,Walk Away,I don't know why she's with me \r\nI only bro...
4,Dean Martin,Happy Feet,Sound of rain on the window pain \r\nMakes a ...
5,Andy Williams,Sweet Memories,"My world is like a river, as dark as it is dee..."
6,Stone Temple Pilots,Adhesive,My friend Blue he runs the show with hot pink ...
7,Hank Snow,Blue Velvet Band,"If you listen a story I'll tell you, \r\nOf a..."
8,Dusty Springfield,Crumbs Off The Table,"Ooh, hoo \r\nGet up in the morning \r\n'Bout..."
9,Quiet Riot,Tin Soldiers,I'm... A little tin soldier that wants to jump...


In [9]:
df['text'][0]

"Yeah!  \r\n  \r\nQueen Bitch, eat the rich  \r\nI'm on the second course today.  \r\nI'm not the first  \r\nAnd I won't be the worst  \r\nYou done most of LA.  \r\nCan't find a virgin  \r\nI can get you a surgeon  \r\n24 hours a day.  \r\nWasn't Jean Genie a Rolling Stones hit?  \r\nI'm about to be blown away.  \r\n  \r\nCome on, sing it  \r\n  \r\nTake me to the place where the sunshine flows  \r\nOh my sunset rodeo.  \r\n  \r\nHot fudge  \r\nHere comes the judge  \r\nThere's a green card and a way  \r\nThe holy ghost  \r\nAnd the whole east coast  \r\nAre moving to LA  \r\nAnd we've been  \r\nDreaming of this feeling  \r\nSince 1988  \r\nMother, things have got to change  \r\nI'm moving to LA.  \r\n  \r\nTake the piss  \r\nI was English  \r\nGod bless you Uncle Sam.  \r\nYou got a cool gene pool  \r\nAnd I went to school  \r\nAnd God knows I love to tan.  \r\nMaking cents and dead presidents  \r\nBefore I could count to ten.  \r\nWith the nation behind me  \r\nCan't stop the limey  

In [10]:
df.shape

(5000, 3)

In [11]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [12]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [16]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rishi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [17]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [22]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [23]:
similarity[0]

array([1.        , 0.00931588, 0.00863208, ..., 0.00692858, 0.01607059,
       0.03763853])

In [24]:
df[df['song'] == 'Crying Over You']

,artist,song,text


In [27]:
def recommendation(song_title):
    # Check if the song exists in the DataFrame
    if song_title in df['song'].values:
        # Get the index of the song
        idx = df[df['song'] == song_title].index[0]
        
        # Calculate similarity scores
        distances = sorted(list(enumerate(similarity[idx])), reverse=True, key=lambda x: x[1])
        
        # Get recommended songs (excluding the song itself)
        songs = []
        for m_id in distances[1:21]:  # Top 20 recommendations
            songs.append(df.iloc[m_id[0]].song)
        
        return songs
    else:
        return f"'{song_title}' not found in the dataset."


In [28]:
recommendation('Crying Over You')

"'Crying Over You' not found in the dataset."

In [29]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))